<a href="https://colab.research.google.com/github/MixanBac/HPC2022/blob/main/PI_value_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-1r4r63ik
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-1r4r63ik
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4307 sha256=4226456244434174631351dae6356460f741b9f87662448612186b52dfff6f7d
  Stored in directory: /tmp/pip-ephem-wheel-cache-3x78304l/wheels/ca/33/8d/3c86eb85e97d2b6169d95c6e8f2c297fdec60db6e84cb56f5e
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [4]:
!   nvidia-smi

Thu Nov 17 13:42:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Как мы видим, Google Colab использует видеокарту Nvidia Tesla T4

In [10]:
%%cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <iostream>
#include <stdio.h>
#include <ctime>
#include <vector>


using namespace std;

clock_t c_start, c_end;
int n = 1024 * 1024 * 32; // Количество точек

__global__ void count_pi(float* dev_randX, float* dev_randY, int* dev_threads_num, int n) {   // Ядро, на котором вычисления выполняются параллельно на большом числе нитей
	int tid = threadIdx.x + blockIdx.x * blockDim.x;                                            // Глобальный индекс нити

	int cont = 0;
	for (int i = tid * 128; i < 128 * (tid + 1); i++) {
		if (dev_randX[i] * dev_randX[i] + dev_randY[i] * dev_randY[i] < 1.0f) { // Условие нахождения точки внутри единичной окружности
			cont++;
		}
	}
	dev_threads_num[tid] = cont;
}

int main() {

	vector<float> randX(n);                   // Инициализация координаты X
	vector<float> randY(n);                   // Инициализация координаты Y

	srand((unsigned)time(NULL));              // Функция srand выполняет инициализацию генератора случайных чисел rand
	for (int i = 0; i < n; i++) {
		randX[i] = float(rand()) / RAND_MAX;    // Генерация координаты X(делим на RAND_MAX, чтобы числа были от 0 до 1)
		randY[i] = float(rand()) / RAND_MAX;    // Генерация координаты Y
	}

	c_start = clock();                        // Начало подсчета времени
	int c_count = 0;                          // Переменная хранящая в себе количество точек, находящихся внутри единичной окружности

	for (int i = 0; i < n; i++) {
		if (randX[i] * randX[i] + randY[i] * randY[i] < 1.0f) { // Условие нахождения точки внутри единичной окружности
			c_count++;                                            // При выполнении условия - засчитываем данную точку как находящуюся внутри единичной окружности
		}
	}
	c_end = clock();                                         // Конец подсчета времени
	float t_cpu = (float)(c_end - c_start) / CLOCKS_PER_SEC; // Разделив общее количество тиков на число тиков в секунду мы получаем количество секунд
	float c_num = float(c_count) * 4.0 / n;                  // Подсчет числа PI
	cout << "CPU Time" << endl;                              // Выводы
	cout << "PI_value = " << c_num << endl;
	cout << "time= " << t_cpu * 1000 << " ms" << endl;

	cudaEvent_t start, stop;
	cudaEventCreate(&start);     // Создание событий
	cudaEventCreate(&stop);
	cudaEventRecord(start, 0);   // Запись события

	size_t size = n * sizeof(float);
	float* dev_randX;
	float* dev_randY;
	cudaMalloc((void**)&dev_randX, size);     // Выделение памяти на GPU
	cudaMalloc((void**)&dev_randY, size);

	cudaMemcpy(dev_randX, &randX.front(), size, cudaMemcpyHostToDevice); // Копирование данных между хостом и устройством
	cudaMemcpy(dev_randY, &randY.front(), size, cudaMemcpyHostToDevice);

	int threadsPerBlock = 512;                                           // Задание конфигурации запуска нитей
	int block_num = n / (128 * threadsPerBlock);
	int* dev_threads_num;
	cudaMalloc((void**)&dev_threads_num, n / 128 * sizeof(int));         // Выделение памяти на GPU

	count_pi << <block_num, threadsPerBlock >> > (dev_randX, dev_randY, dev_threads_num, n);  // Запуск ядра

	int* threads_num = new int[n / 128];
	cudaMemcpy(threads_num, dev_threads_num, n / 128 * sizeof(int), cudaMemcpyDeviceToHost);  // Копирование данных между устройством и хостом

	int g_count = 0;
	for (int i = 0; i < n / 128; i++) {
		g_count += threads_num[i];                // Посчет числа точек внутри единичной окружности
	};

	cudaEventRecord(stop, 0);   //Запись события
	cudaEventSynchronize(stop);
	float t_gpu;
	cudaEventElapsedTime(&t_gpu, start, stop);  // Вычисляет время, прошедшее между событиями
	cudaEventDestroy(start);                    // Уничтожает объект события
	cudaEventDestroy(stop);                     // Уничтожает объект события

	float g_num = float(g_count) * 4.0 / n; // Подсчет числа PI после расчета на GPU
	cout << "GPU Time" << endl;             // Выводы
  cout << "PI_value = " << g_num << endl;
	cout << "time = " << t_gpu << " ms" << endl;
}

CPU Time
PI_value = 3.14168
time= 520.823 ms
GPU Time
PI_value = 3.14168
time = 70.8668 ms



Мы можем наблюдать, что вычисления на GPU происходят быстрее, чего и следовало ожидать.